In [38]:
import pandas as pd
import numpy as np
import os

os.chdir("/Users/burke/Documents/research/scrooge")

oneDocscreeningServices = pd.read_excel("SingleProvider10YearLifetimeScreeningServices.xlsx")
oneDocscreeningServices['year'] = oneDocscreeningServices.visitDate.dt.year

oneDocPanel = pd.read_excel("SingleProvider10YearPanel.xlsx")
oneDocVisits = pd.read_excel("SingleProvider10YearLifetimeVisits.xlsx")
oneDocVisits['year'] = oneDocVisits.visitDate.dt.year

lastYearScreening = oneDocscreeningServices[oneDocscreeningServices.year==2026]
lastYearVisits = oneDocVisits[oneDocVisits.year==2026]

In [44]:
print len(lastYearVisits)
print len(lastYearScreening)
print lastYearScreening['timeSpent'].sum()

3464
7908
16650.45


In [49]:
lastYearScreening.groupby("Service")['timeSpent'].

AttributeError: Cannot access callable attribute 'sort_values' of 'SeriesGroupBy' objects, try using the 'apply' method